In [1]:
import json
import re
import pandas as pd
import urllib
from urllib.request import Request
from bs4 import BeautifulSoup

In [2]:
import urllib.request
from bs4 import BeautifulSoup

url = "http://www.alltime-athletics.com/m_100ok.htm"

with urllib.request.urlopen(url) as resp:
    processed_page = BeautifulSoup(resp.read(), "html.parser")

pre_tag = processed_page.find("pre")
values_text = pre_tag.text

# Split the text into individual lines
lines = values_text.split("\n")

values_array = []

for line in lines:
    # Remove any leading/trailing whitespaces
    line = line.strip()
    if line:
        # Split the line into values based on the pattern you provided
        values = line.split()

        # Check if the 3rd index contains a valid wind category
        wind_category = values[2]
        if wind_category[0] in ['+', '-', '±'] and wind_category[1:].replace('.', '').isdigit():
            # Combine the strings after wind and before country code
            name = ' '.join(values[3:values.index(values[5])])

            # Replace the original values with the combined name
            values = values[:3] + [name] + values[5:]

        if len(values[4]) != 3:
            # Concatenate the 3rd index with the 2nd index
            values[3] = values[3] + ' ' + values[4]
            del values[4]
        if not values[4].isupper():
            values[3] = values[3] + ' ' + values[4]
            del values[4]

        # Delete the value in the 6th index
        if len(values) > 6:
                del values[6]

        values_array.append(values)


# Join values after 5th index but before the last index in each row
for row in values_array:
    if len(row) > 7:
        joined_values = ' '.join(row[6:-1])
        row[6] = joined_values
        del row[7:-1]



for row in values_array:
    print(row)

['1', '9.58', '+0.9', 'Usain Bolt', 'JAM', '21.08.86', 'Berlin', '16.08.2009']
['2', '9.63', '+1.5', 'Usain Bolt', 'JAM', '21.08.86', 'London', '05.08.2012']
['3', '9.69', '±0.0', 'Usain Bolt', 'JAM', '21.08.86', 'Beijing', '16.08.2008']
['3', '9.69', '+2.0', 'Tyson Gay', 'USA', '09.08.82', 'Shanghai', '20.09.2009']
['3', '9.69', '-0.1', 'Yohan Blake', 'JAM', '26.12.89', 'Lausanne', '23.08.2012']
['6', '9.71', '+0.9', 'Tyson Gay', 'USA', '09.08.82', 'Berlin', '16.08.2009']
['7', '9.72', '+1.7', 'Usain Bolt', 'JAM', '21.08.86', 'New York City', '31.05.2008']
['7', '9.72', '+0.2', 'Asafa Powell', 'JAM', '23.11.82', 'Lausanne', '02.09.2008']
['9', '9.74', '+1.7', 'Asafa Powell', 'JAM', '23.11.82', 'Rieti', '09.09.2007']
['9', '9.74', '+0.9', 'Justin Gatlin', 'USA', '10.02.82', 'Ad-Dawhah', '15.05.2015']
['11', '9.75', '+1.1', 'Yohan Blake', 'JAM', '26.12.89', 'Kingston', '29.06.2012']
['11', '9.75', '+1.5', 'Yohan Blake', 'JAM', '26.12.89', 'London', '05.08.2012']
['11', '9.75', '+0.9', '

In [3]:
frame = pd.DataFrame.from_dict(values_array)
frame

,0,1,2,3,4,5,6,7
0,1,9.58,+0.9,Usain Bolt,JAM,21.08.86,Berlin,16.08.2009
1,2,9.63,+1.5,Usain Bolt,JAM,21.08.86,London,05.08.2012
2,3,9.69,±0.0,Usain Bolt,JAM,21.08.86,Beijing,16.08.2008
3,3,9.69,+2.0,Tyson Gay,USA,09.08.82,Shanghai,20.09.2009
4,3,9.69,-0.1,Yohan Blake,JAM,26.12.89,Lausanne,23.08.2012
...,...,...,...,...,...,...,...,...
3983,3575,10.09,+0.8,Benjamin Azamati,GHA,14.01.98,Montreuil,31.05.2023
3984,3575,10.09,±0.0,Trayvon Bromell,USA,10.07.95,Firenze,02.06.2023
3985,3575,10.09,±0.0,Akani Simbine,RSA,21.09.93,Firenze,02.06.2023
3986,3575,10.09,+0.5,J.T Smith,USA,12.01.98,Nashville,03.06.2023


In [4]:
import csv

filename = 'men_100m.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'Place',
        'Time',
        'Wind',
        'Athlete',
        'Country',
        'Date of Birth',
        'City',
        'Event Date'
    ]
    writer = csv.writer(csvfile)
    
    writer.writerow(fieldnames)  # Write the header row
    
    for row in values_array:
        # Check if there are multiple strings between the date and the weird number        
        writer.writerow(row)
    
print(f"Data saved to {filename} successfully.")

Data saved to men_100m.csv successfully.
